### Mounting Azure Data Lake storage on databricks

In [0]:
storage_account_name = "ecommanvith"
container_name = "landing-zone-2"
account_key = "F6qp1g+E+eY87cnuXkuhz/3O3WnNF2q+TBtKRUOhskEmqFqzGiaWua38K+6vEXELczr/wAN4T33v+AStUT9RYw==" 
mount_point = "/mnt/landing-zone-2"

# Mount the Data Lake container using wasbs scheme and correct configuration
dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point=mount_point,
    extra_configs={
        f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": account_key
    }
)

print(f"Mounted {container_name} to {mount_point} successfully.")


Mounted landing-zone-2 to /mnt/landing-zone-2 successfully.


In [0]:
%fs ls /mnt/landing-zone-2

path,name,size,modificationTime
dbfs:/mnt/landing-zone-2/buyers-raw-2/,buyers-raw-2/,0,0
dbfs:/mnt/landing-zone-2/countries-raw-2/,countries-raw-2/,0,0
dbfs:/mnt/landing-zone-2/sellers-raw-2/,sellers-raw-2/,0,0
dbfs:/mnt/landing-zone-2/users-raw-2/,users-raw-2/,0,0


### Starting spark Session

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("Ecom-DataPipeline").getOrCreate()  

In [0]:
spark

### Accessing the parquet files and creating a delta table

In [0]:
userDf=spark.read.format("parquet")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/mnt/landing-zone-2/users-raw-2/")

In [0]:
userDf.show(5)

+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|   country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+-

In [0]:
userDf.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/users")                
    

In [0]:
buyersDf=spark.read.format("parquet")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("/mnt/landing-zone-2/buyers-raw-2/")     




In [0]:
buyersDf.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/buyers")

In [0]:
countriesDf=spark.read.format("parquet")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("/mnt/landing-zone-2/countries-raw-2/")

In [0]:
countriesDf.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/countries")

In [0]:
sellersDf=spark.read.format("parquet")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("/mnt/landing-zone-2/sellers-raw-2/")

In [0]:
sellersDf.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/sellers")